In [15]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import binascii

In [16]:
%run '../catalog_common.ipynb'

In [17]:
base_dir = os.getcwd()
slac_dir = os.path.join(base_dir,'slac')
data_dir = os.path.join(slac_dir, 'data')
temp_file = os.path.join(slac_dir, 'temp.pt')

if not os.path.exists(slac_dir):
        os.mkdir(slac_dir)
if not os.path.exists(data_dir):
        os.mkdir(data_dir)


batch_size=128

In [18]:
def slac_list_onrl(catalog): 
    onrl_scope = "edu.onrl.slac_collab"
    onrl_type = "model"
    text_entries = catalog.list_all()
#     item_list = my_parse(text_entries)
#     obj_list = [json.loads(item) for item in item_list]
    obj_list = json.loads(text_entries)
    return obj_list


def slac_list_onrl_models(obj_list):
    mdata_list = []
    for item in obj_list:
        if item["metadata"] is not None:
            mdata_list.append(item["metadata"])
#     mdata_list =[item["metadata"] for item in obj_list if item["metadata"] is not None]
    names=[item["name"] for item in mdata_list]
    return names


def convert_2_model(encoded_str):
    new_bin=binascii.unhexlify(encoded_str)
    save_to_file(temp_file, new_bin)
    model = torch.jit.load(temp_file)
    model.eval()
    return model


def slac_search_onrl_models(catalog, name):
    obj_list = slac_list_onrl(catalog)
    obj_list_2 = [item for item in obj_list if item["metadata"] is not None]
    item_list = [item for item in obj_list_2 if item["metadata"]["name"].strip() == name.strip()]
    if len(item_list) > 0:
        match = item_list[0]
        if match is not None: 
            answer= match["data_uri"]
            return convert_2_model(answer)

        
def print_catalog_models(catalog):
    obj_list = slac_list_onrl(catalog)
    names = slac_list_onrl_models(obj_list)
    print("Current Models in the Catalog")
    print(names)    

In [10]:
def test(model, device, test_data): 
    model = model.to(device)
    test_loader=DataLoader(test_data,shuffle=False,batch_size=batch_size)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))